# API 연습

## github서버에서 가져오기

In [1]:
#get매서드의 arguments로 첫번째는 URL, 두번째는 keyword방식으로 headers값을 설정해주었음
#status_code를 통해 서버와의 연결상태 확인(200은 정상,https://httpstatuses.com/200에 들어가서 200만 바꿔주면 오류 파악이 가능하다)
#parameter를 주지 않으면 요청에 대해 처리할수 없음(422)반환
import requests

response = requests.get('https://api.github.com/repositories',
                        headers={'Accept':'application/vnd.github.v3+json'})
print(response.status_code)

200


In [2]:
#response에 대한 요소 확인
print(f"인코딩: {response.encoding}")
print(f"콘텐츠 타입: {response.headers['Content-Type']}")
print(f"서버: {response.headers['server']}")

인코딩: utf-8
콘텐츠 타입: application/json; charset=utf-8
서버: GitHub.com


In [5]:
#json으로 파싱작업을 통해 일련의 문자열을 토큰으로 분해하고 이들로 이루어진 파스 트리를 만들어줌
#컨텐츠 타입이 json ==> json으로 변환 가능한 객체 ==>
# ==> json라이브러리를 활용하여 구조화(딕셔너리형태의 문자열(Seriealization문자열)을 딕셔너리화해 반환)
import json
print(json.dumps(response.json()[0], indent=2)[:200])

{
  "id": 1,
  "node_id": "MDEwOlJlcG9zaXRvcnkx",
  "name": "grit",
  "full_name": "mojombo/grit",
  "private": false,
  "owner": {
    "login": "mojombo",
    "id": 1,
    "node_id": "MDQ6VXNlcjE=",



In [20]:
print(type(response.json()))

<class 'dict'>


In [ ]:
#public repositories에 대한 정보 전부(너무 많아 수행X)
response.json()

In [15]:
#검색API를 사용해보자
#parameter를 주지 않으면 요청에 대해 처리할수 없음(422)반환 (통신에 대한 query parameters는 API마다 다름)
response = requests.get('https://api.github.com/search/repositories')
print(response.status_code)

422


In [16]:
#(언어는 python)으로 data_science관련을 검색, headers
response = requests.get('https://api.github.com/search/repositories',
                       params={'q':'data_science+language:python'},
                       headers={'Accept':'application/vnd.github.v3.text-match+json'})
print(response.status_code)

200


In [21]:
response.json().keys()

dict_keys(['total_count', 'incomplete_results', 'items'])

In [22]:
#response의 key중에 items(repository에 대한 정보)의 첫번째에서 text_match라는 키값의 value 호출
response.json()['items'][0]['text_matches']

[{'object_url': 'https://api.github.com/repositories/26382146',
  'object_type': 'Repository',
  'property': 'description',
  'fragment': 'code for Data Science From Scratch book',
  'matches': [{'text': 'Data Science', 'indices': [9, 21]}]},
 {'object_url': 'https://api.github.com/repositories/26382146',
  'object_type': 'Repository',
  'property': 'name',
  'fragment': 'data-science-from-scratch',
  'matches': [{'text': 'data', 'indices': [0, 4]}]}]

In [23]:
#items는 30개의 리스트를 value로 가짐
len(response.json()['items'])

30

## pytorch오픈소스 가져오기

pagination = 서버의 과부하를 방지하기 위해 서버에서 응답 개수를 제한  
next 30개만 보여줬음

In [25]:
response = requests.get('https://api.github.com/repos/pytorch/pytorch/issues',
                        headers={'Accept':'application/vnd.github.v3.text-match+json'})
print('Response Code', response.status_code)
print('Number of comments', len(response.json()))

Response Code 403
Number of comments 2


In [27]:
response.links

{}

In [26]:
def get_all_pages(url, params=None, headers=None):
    output_json = []
    response = requests.get(url,params=params, headers=headers)
    if response.status_code ==200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next+url is not None:
                output_json += get_all_pages(next_url, params, headers)
    return output_json

In [ ]:
#가져온 데이터 DataFrame화해서 결과확인
import pandas as pd

out = get_all_pages(
    "https://api.github.com/repos/pytorch/pytorch/issues/comments",
    params={
        'since': '2022-01-01T10:00:01Z',
        'sorted': 'created',
        'direction': 'desc'
    },
    headers={'Accept' : 'application/vnd.github.v3+json'}
)

df = pd.DataFrame(out)
print(df['body'].count())
df[['id', 'created_at','body']].sample(1)